# First thing to do, make a copy of this notebook under File->Make a Copy with some intelligent naming convention.

---

# Set your observing times here.

## NB: This will be a broader window that you'll actually use. The final output will give you orbit-by-orbit pointing information. So you can use a subset of the orbits that are chosen here.

In [1]:
from datetime import datetime
import numpy as np

# Note that tstart and tend should span for at least one day because of the way the scripts are written.
tstart = '2018-01-12T00:00:00'
tend = '2018-01-13T00:00:00'

ttarget = '2018-01-12T21:54:00'

# Turn these into datetime objects

tstart = datetime.strptime(tstart, '%Y-%m-%dT%H:%M:%S')
tend = datetime.strptime(tend, '%Y-%m-%dT%H:%M:%S')
ttarget = datetime.strptime(ttarget,'%Y-%m-%dT%H:%M:%S')


# Get the Sky PA from external code:

sky_pa = 170.


---
# Step 1, get the nominal RA/Dec Position for the Moon at the start.

## We use this to determine what the occultation times are.

In [2]:
from skyfield.api import Loader
from astropy.time import Time
import astropy.units as u

load = Loader('../../data')

ts = load.timescale()
planets = load('de436.bsp')

astro_time = Time(ttarget)
t = ts.from_astropy(astro_time)
moon, earth = planets['moon'], planets['earth']
astrometric = earth.at(t).observe(moon)
ra, dec, distance = astrometric.radec()


occstring = "./occ {0:0.4f} {1:0.4f} Latest_TLE.txt {2}:{3:03d}:00:00:00 {4}:{5:03d}:00:00:00 lunar_{3}_{5}.occ".format(
      ra.to(u.deg).value, dec.to(u.deg).value,
      tstart.timetuple().tm_year, tstart.timetuple().tm_yday, 
      tend.timetuple().tm_year, tend.timetuple().tm_yday)


outfile = '../orbit_engine/lunar_{0}to{1}.sh'.format(tstart.timetuple().tm_yday, tend.timetuple().tm_yday)
print(outfile)

f = open(outfile, 'w')
f.write(occstring)
f.close()


import os
import stat

st = os.stat(outfile)
os.chmod(outfile, st.st_mode | stat.S_IEXEC)



../orbit_engine/lunar_12to13.sh


---
# Step 2: Go run the code that figures out the unocculted periods for the RA/Dec and the date range reported above.

This works on lif. There are example shell scripts in the ../orbit_engine directory that use the version that Karl already compiled for the nuops users.

First, get the latest TLE archive:

`./get_latest_TLE.sh`

Run the script that was produced above.


# Step 3: Initialize your libraries and parse the resulting occultation file:


In [3]:
from nustar_planning import io
occfile= "../orbit_engine/lunar_{0}_{1}.occ".format(tstart.timetuple().tm_yday,tend.timetuple().tm_yday )

#print(occfile)
orbits = io.parse_occ(occfile)

# NB: The "head" command here only shows the first couple of rows. Do a "print(orbits)" to see them all.
orbits.head()


,visible,occulted
0,2018-01-12 00:58:23,2018-01-12 01:54:33
1,2018-01-12 02:35:02,2018-01-12 03:31:12
2,2018-01-12 04:11:41,2018-01-12 05:07:52
3,2018-01-12 05:48:20,2018-01-12 06:44:31
4,2018-01-12 07:24:59,2018-01-12 08:21:10


In [4]:
import nustar_planning.lunar_planning as lunar_planning

In [5]:
# Sanity check to make sure you havde the correct orbit(s)

sub = orbits.iloc[13:].reset_index()
lunar_planning.position(sub, steps = 5, parallax_correction=True)

Aim Time            RA         Dec
2018-01-12T21:54:50 247.14594  -16.53041
2018-01-12T22:06:04.600000 247.20129  -16.77891
2018-01-12T22:17:19.200000 246.77068  -17.00746
2018-01-12T22:28:33.800000 246.12057  -17.10167
2018-01-12T22:39:48.400000 245.64668  -17.02015


In [7]:
# PA == 0 (default), Rmoon = 940 arcseconds (default), 6 arcminutes maximum drift (default) and
# no minimum dwell period (default)
outfile = 'lunar_{0}_{1}_pointing.txt'.format(tstart.timetuple().tm_yday,tend.timetuple().tm_yday )
lunar_planning.position_shift(sub, parallax_correction=True, pa = 170.0*u.deg, dt = 3*u.s,
                              outfile=outfile, load_path ='../data')
print("Output is stored in: {}".format(outfile))

[#################################] 100% de436.bsp
[#################################] 100% deltat.data
[#################################] 100% deltat.preds
[#################################] 100% Leap_Second.dat


2018:012:21:49:50 RA: 247.00491  Dec: -16.22364

2018:012:21:52:05 RA: 247.09720  Dec: -16.27230

2018:012:21:54:47 RA: 247.17222  Dec: -16.34284

2018:012:21:58:14 RA: 247.19450  Dec: -16.44060

2018:012:22:02:35 RA: 247.14696  Dec: -16.53001

2018:012:22:06:26 RA: 247.06873  Dec: -16.59799

2018:012:22:09:26 RA: 246.97877  Dec: -16.65150

2018:012:22:11:56 RA: 246.88338  Dec: -16.69504

2018:012:22:14:08 RA: 246.78552  Dec: -16.73094

2018:012:22:16:08 RA: 246.68457  Dec: -16.76122

2018:012:22:18:02 RA: 246.58060  Dec: -16.78669

2018:012:22:19:53 RA: 246.47578  Dec: -16.80732

2018:012:22:21:41 RA: 246.37207  Dec: -16.82320

2018:012:22:23:26 RA: 246.26525  Dec: -16.83503

2018:012:22:25:14 RA: 246.16002  Dec: -16.84218

2018:012:22:27:02 RA: 246.05536  Dec: -16.84458

2018:012:22:28:53 RA: 245.94826  Dec: -16.84157

2018:012:22:30:53 RA: 245.84257  Dec: -16.83200

2018:012:22:33:02 RA: 245.73911  Dec: -16.81391

2018:012:22:35:26 RA: 245.63866  Dec: -16.78253

2018:012:22:38:20 RA

In [ ]:
df3 = pd.read_csv('lunar_20171023_v3.txt', sep='\s+', skiprows=1, header=None, names=['Date', 'RA', 'Dec'])
df4 = pd.read_csv('lunar_20171023_v4.txt', sep='\s+', skiprows=1, header=None, names=['Date', 'RA', 'Dec'])



In [ ]:
from astropy.coordinates import SkyCoord

In [ ]:
skypos3 = SkyCoord(df3.RA, df3.Dec,  unit="deg")
skypos4 = SkyCoord(df4.RA, df4.Dec,  unit="deg")



In [ ]:
for ind, pos3 in enumerate(skypos3):
    diff = pos3.separation(skypos4[ind])
    print(diff.arcsec)
    